In [105]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306

In [83]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi) #pass command to the engine

In [84]:
def run_sql(sql, engine): #sql is a string
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result



In [85]:
tables = run_sql("show tables;", engine) #list all the table in the cell
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


In [86]:
chefdetails_query = """
    SELECT *
    FROM judges;
"""
chefdetails_df = df_select(chefdetails_query, engine)
chefdetails_df.head()


Retrieved 755 records


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


In [87]:
#Problem 1 Exclude "Master" and "Canada 6"
chefdetails_query = """
    SELECT *
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada%';
"""

chefdetails_df = df_select(chefdetails_query, engine)
chefdetails_df.head()

Retrieved 322 records


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female


In [88]:
#Problem 2 Trim your chefdetails and judges tables
chefdetails_query = """
    SELECT season, seasonNumber, name, placement, gender
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada%';
"""

chefdetails_df = df_select(chefdetails_query, engine)
chefdetails_df.head()

Retrieved 322 records


,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


In [89]:
judges_query = """
    SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC
    FROM judges
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada%';
"""

judges_df = df_select(judges_query, engine)
judges_df.head()

Retrieved 633 records


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


In [90]:
#Problem 3 Table for 20 remaining different seasons, numbered from 1 to 20
sql_seasons = """
SELECT DISTINCT season, seasonNumber
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6'
ORDER BY seasonNumber;
"""

seasons_df = df_select(sql_seasons, engine)
seasons_df

Retrieved 20 records


,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


In [91]:
#Problem 4a Number of mae and female among all chef contestants 
chef_gender_query = """
    SELECT gender, COUNT(*) as count
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada%'
    GROUP BY gender;
"""

chef_gender_df = df_select(chef_gender_query, engine)
chef_gender_df

Retrieved 2 records


,gender,count
0,Male,170
1,Female,152


In [92]:
#Problem 4b Number of male and female among all winners
winners_gender_query = """
    SELECT gender, COUNT(*) as count
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada%'
      AND placement = 1
    GROUP BY gender;
"""

winners_gender_df = df_select(winners_gender_query, engine)
winners_gender_df

Retrieved 2 records


,gender,count
0,Male,14
1,Female,6


In [93]:
#Problem 4c Number of male and female among all sets of top 3 finalists
top3_gender_query = """
    SELECT gender, COUNT(*) as count
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%'
      AND season NOT LIKE '%Canada 6%'
      AND placement IN (1, 2, 3)
    GROUP BY gender;
"""

top3_gender_df = df_select(top3_gender_query, engine)
top3_gender_df

Retrieved 2 records


,gender,count
0,Male,36
1,Female,25


In [94]:
#Problem 4d Compare the number of times a femae was plaed in the top 3 with the number of times a female won
female_winners = winners_gender_df[winners_gender_df['gender'] == 'Female']['count'].values[0]
female_top3_finalists = top3_gender_df[top3_gender_df['gender'] == 'Female']['count'].values[0]

# Calculate the ratio of female winners to female top 3 finalists
ratio_female_winners_to_top3 = female_winners / female_top3_finalists

# Display the results
print(f"Number of Female Winners: {female_winners}")
print(f"Number of Female Top 3 Finalists: {female_top3_finalists}")
print(f"Ratio of Female Winners to Female Top 3 Finalists: {ratio_female_winners_to_top3}")

Number of Female Winners: 6
Number of Female Top 3 Finalists: 25
Ratio of Female Winners to Female Top 3 Finalists: 0.24


In [107]:
merged_df = pd.merge(chefdetails_df, judges_df, on=['season', 'seasonNumber'], how='inner')

# Task 2: Calculate the proportion of female contestants who placed in the top 3
female_top3_count = merged_df[(merged_df['gender'] == 'Female') & (merged_df['placement'].isin([1, 2, 3]))].shape[0]
total_female_count = merged_df[merged_df['gender'] == 'Female'].shape[0]
female_top3_proportion = female_top3_count / total_female_count

# Task 3: Calculate the proportion of female winners
female_winner_count = merged_df[(merged_df['gender'] == 'Female') & (merged_df['placement'] == 1)].shape[0]
female_winner_proportion = female_winner_count / total_female_count

# Task 4: Compare the proportions
print(f"Proportion of females in top 3: {female_top3_proportion:.2%}")
print(f"Proportion of female winners: {female_winner_proportion:.2%}")

Proportion of females in top 3: 15.98%
Proportion of female winners: 3.87%


In the dataset, we observed that therea were a total of 245 female chefs who reached the top 3 finalsts, out of which only 6 went on to win the competition. The ratio of female winnners to demale top 3 finalists is approximately 0.24 (6/25), indicating that for every four female chefs who reached the top 3, one went on to win.

The proportions provide additional insights. The proportion of females in the top 3 is 15.98%, while the proportion of female winners is 3.87%. This suggests that women who place in the top 3 are less likely than expected to ultimately win the competition.

The relatively low ratio (0.24) and the difference in proportions between females in the top 3 (15.98%) and female winners (3.87%) suggest that women who plave in the top 3 are less likely than expected to ultimately win the competition. In other words, while a reasonable number of femae chefs make it to the top 3, a smaller perentage of them go on to win.

In [108]:
female_top3_finalists
female_winners
total_female_chefs=chef_gender_df[chef_gender_df['gender'] == 'Female']['count'].values[0]
total_female_chefs
contingency_table = [[female_top3_finalists, total_female_chefs - female_top3_finalists],
                     [female_winners, total_female_chefs - female_winners]]
# Perform a Chi-squared test
chi2, p, _, _ = stats.chi2_contingency(contingency_table)

print(f"Chi-squared value: {chi2}")
print(f"P-value: {p}")

# Check if the p-value is less than the significance level (e.g., 0.05) to reject the null hypothesis
if p < 0.05:
    print("Reject the null hypothesis: There is a significant difference.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference.")

Chi-squared value: 11.638426090038992
P-value: 0.0006460324212308851
Reject the null hypothesis: There is a significant difference.


In the hypothesis, we can set that the null hypothesis and althernative hypothesis as the following:
Null hypothesis: There is no significant difference between the proportion of female contestants who reach the top 3 and the proportion of femake contestnants who ultimately win the conpetition.

Alternative hypothesis: The proportion of female contestnats who reach the top 3 is significantly different from the proportion of female contestants who ultimately win the competition.

We can use the statistical test to compare the proportions, such as the Chi-squared test for independence or a z-test for proportions. The test can help us to determin of any obseved dfferenes between the proportions are statistically significant or if they could have ocurred by chance.

From the above test result, the Chi-squared value is 11.64 and the p-value is 0.000646 which is lower than the significance leve of 0.05. This indicates that the observed difference in proportions is unlikely to beue to random chance alone.

Based on the p-value bein gbelow the singiciant level, we have enough evidence to reject the null hypothesis. This suggests that there is a significant diference between the proportion of female contestants who reach the top 3 and the proportion of female contestants who ultimately win the competition.



In [96]:
# Task 5 Make a table with one row for each first place winner 
# Filter judges_df
guest_judge = judges_df[judges_df['competedOnTC'] == 'Yes']

# Extract unique guest judges
chef_judge_list = guest_judge['guestJudge'].unique()

# Select relevant chef details
chef_info = df_select("""
    SELECT season, seasonNumber, name
    FROM (
        SELECT placement, name, season, seasonNumber
        FROM chefdetails
        WHERE placement = 1
    ) AS subquery
    WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';
""", engine)

# Create 'guest_judge?' column
chef_info['guest_judge?'] = chef_info['name'].isin(chef_judge_list).map({True: 'Yes', False: 'No'})

chef_info

Retrieved 20 records


,season,seasonNumber,name,guest_judge?
0,All Stars: New York,8,Richard Blais,Yes
1,All-Stars L.A.,17,Melissa King,Yes
2,Boston,12,Mei Lin,No
3,California,13,Jeremy Ford,No
4,Charleston,14,Brooke Williamson,Yes
5,Chicago,4,Stephanie Izard,Yes
6,D.C.,7,Kevin Sbraga,No
7,Colorado,15,Joseph Flamm,No
8,Houston,19,Kah-wai Lo,No
9,Kentucky,16,Kelsey Barnard Clark,Yes


In [97]:
#Task 6 Find all contestants who were not first place winners

chef_not_win = df_select("""
    SELECT subquery.name, MIN(subquery.season) AS season, MIN(subquery.seasonNumber) AS seasonNumber
    FROM (
        SELECT DISTINCT name, season, seasonNumber
        FROM chefdetails
        WHERE placement != 1
    ) AS subquery
    INNER JOIN judges ON subquery.name = judges.guestJudge
    WHERE subquery.season NOT LIKE '%Canada%' AND subquery.season NOT LIKE '%Master%'
    GROUP BY subquery.name;
""", engine)

chef_not_win

Retrieved 32 records


,name,season,seasonNumber
0,Brooke Williamson,Seattle,10
1,Richard Blais,Chicago,4
2,Antonia Lofaso,All Stars: New York,4
3,Carlos Gaytan,New Orleans,11
4,Nicole Gomes,World All Stars,20
5,Sam Talbot,Charleston,2
6,Bryan Voltaggio,All-Stars L.A.,6
7,Mike Isabella,All Stars: New York,6
8,Spike Mendelsohn,All Stars: New York,4
9,Dawn Burrell,Portland,18
